In [1]:
# En-tete

import os
os.chdir("/cal/homes/arenault/workspace/PAF_ScrapIngress")
import csv
import math as m
import numpy as np
import cv2

In [2]:
    # Dichotomie 2D

    # La fonction prend en entree les limites de la zone initiale, la longueur de la zone finale, les coordonnées [x,y] du portail.
    # Elle renvoie l'ID de la zone dans laquelle se trouve le portail; -1 si le portail n'est pas dans la zone scrappee.

    def dichotomie2D(xmin,xmax,ymin,ymax,precision,portail):
        xp,yp = portail[0],portail[1]
        ID = 0
        xc,yc = (xmax-xmin)/2+xmin,(ymax-ymin)/2+ymin 
        if (xp > xmax or xp < xmin or yp > ymax or yp < ymin):       # Le portail est hors limite, on renvoie donc -1
            return -1
        while (ymax-ymin > precision and xmax-xmin > precision):     # Le portail etant dans les limites, on effectue une dichotomie
            if (xp <= xc and yp >= yc):                              # Portail est dans la zone superieure gauche
                ID = 10*ID+1
                xmax,ymin = xc,yc
                xc,yc = xc-(xmax-xmin)/2,yc+(ymax-ymin)/2
            elif (xp > xc and yp > yc):                              # Portail est dans la zone superieure droite
                ID = 10*ID+2
                xmin,ymin = xc,yc
                xc,yc = xc+(xmax-xmin)/2,yc+(ymax-ymin)/2
            elif (xp < xc and yp < yc):                              # Portail est dans la zone inferieure gauche
                ID = 10*ID+3
                xmax,ymax = xc,yc
                xc,yc = xc-(xmax-xmin)/2,yc-(ymax-ymin)/2
            elif (xp >= xc and yp <= yc):                            # Portail est dans la zone inferieure droite
                ID = 10*ID+4
                xmin,ymax = xc,yc
                xc,yc = xc+(xmax-xmin)/2,yc-(ymax-ymin)/2
        return ID

In [3]:
# Test dichotomie2D

xmin,xmax,ymin,ymax = 0,4,0,4
precision = 1
portails = [[i+0.5,j+0.5] for i in range(4) for j in range(4)]
portails.append([0,7])

[dichotomie2D(xmin,xmax,ymin,ymax,precision,p) for p in portails]

[33, 31, 13, 11, 34, 32, 14, 12, 43, 41, 23, 21, 44, 42, 24, 22, -1]

In [4]:
# Conversion de la latitude, longitude en coordonnées cartésiennes

# La fonction prend en entree latitude et longitude d'un point de référence (l'origine de notre repère) ainsi que la
# latitude et la longitude du point qui nous interesse.
# En sortie on obtient les coordonnees (x,y) de ce point par rapport à l'origine en entrée.
# on utilise la formule "Haversine"

# Rq: la latitude correspond au y et la longitude au x du repere cartesien.

def degreToRad(deg):
    return deg*m.pi/180

def conversion(origin_lat,origin_long,latitude,longitude):
    origin_lat,origin_long = degreToRad(origin_lat),degreToRad(origin_long) 
    latitude, longitude = degreToRad(latitude),degreToRad(longitude) 
    dlat = latitude-origin_lat
    dlong = longitude-origin_long
    a = m.sin(dlat/2)**2 + m.cos(origin_lat)*m.cos(latitude)*m.sin(dlong/2)**2
    c = 2*m.asin(a**(0.5))
    radius = 6371   #rayon de la Terre en km
    if (latitude < origin_lat or longitude < origin_long):  #certaines distances sont negatives
        return -c*radius
    return c*radius


In [5]:
# ajout de l'id geographique dans le fichier ficIn


# La fonction prend en entree le fichier dans lequel il faut ajouter l'id, les coordonnees GPS du point origine,
# les coordonnees GPS extremes en x et en y, la precision (en km).

fichier_a_lire = 'portails.csv'

def id_geographique(fichier, o_lat,o_lon,x_lon,y_lat,precision):
    xmin = 0
    xmax = conversion(o_lat,o_lon,o_lat,x_lon)
    ymin = 0
    ymax = conversion(o_lat,o_lon,y_lat,o_lon)
    with open(fichier,'w') as fic:
        with open(fichier_a_lire, 'r') as to_read:
            csv_reader = csv.reader(to_read, delimiter = ';')
            csv_writer = csv.writer(fic, delimiter = ';')
            for line in csv_reader:          
                portal = line[0].split(',')                            # portal[lat,lon]
                x,y = conversion(o_lat,o_lon,o_lat,float(portal[1])),conversion(o_lat,o_lon,float(portal[0]),o_lon)
                portal[0],portal[1] = x,y                              # portal[x,y]
                ID = dichotomie2D(xmin,xmax,ymin,ymax,precision,portal)
                csv_writer.writerow(line + [ID])

In [10]:
# Generation d'un fichier test

ficIn = "./portails_1000.csv"
ficOut = "./test_id.csv"

with open(ficIn, 'r') as ficIn:
    with open(ficOut, 'w') as ficOut:
        csv_reader = csv.reader(ficIn, delimiter = ';')
        csv_writer = csv.writer(ficOut, delimiter = ';')
        k = 1000
        for line in csv_reader:
            csv_writer.writerow(line)
            if (k==1):
                break
            k -= 1
            
id_geographique('./test_id2.csv',42.0, -5.0,8.0,52.0,1)  # on créé des zones de 1km² sur la France

In [18]:
# Visualisation des résultats
    

with open('test_id2.csv', 'r') as to_read:
    csv_r = csv.reader(to_read, delimiter = ';')
    nb_par_zone = {}
    for line in csv_r:
        zone = line[4][:]                       # la precision donne une zone de 1km² a chaque chiffre tronqué on multiplie par 4 la surface
        if not zone in nb_par_zone:
            nb_par_zone[zone] = (line[0], 1)
        else:
            nb_par_zone[zone] = (nb_par_zone[zone][0], nb_par_zone[zone][1]+1)

temp = nb_par_zone.values()
temp.sort(key=lambda zone : zone[1], reverse = True)
temp

[('48.863091,2.332792', 28),
 ('45.754493,4.827342', 25),
 ('48.872749,2.352206', 21),
 ('48.869972,2.352322', 21),
 ('48.855786,2.356904', 19),
 ('47.08086,8.44014', 19),
 ('46.950787,7.440888', 18),
 ('48.865657,2.327775', 17),
 ('48.871047,2.34502', 15),
 ('48.871186,2.336074', 15),
 ('43.694427,7.279027', 14),
 ('48.871088,2.346421', 14),
 ('48.811752,2.384771', 13),
 ('48.890586,2.247773', 13),
 ('48.887755,2.250477', 13),
 ('46.228736,6.081585', 11),
 ('46.522899,6.631997', 10),
 ('45.753447,4.828552', 10),
 ('47.900246,1.901121', 9),
 ('48.893434,2.329654', 8),
 ('46.993628,6.942365', 8),
 ('48.870039,2.393388', 8),
 ('48.860589,2.367598', 7),
 ('47.899624,1.902164', 7),
 ('48.863962,2.334041', 7),
 ('45.750747,4.839187', 7),
 ('48.868631,2.335426', 7),
 ('45.746587,4.846252', 6),
 ('43.615722,3.874201', 6),
 ('48.889481,2.326119', 6),
 ('48.866457,2.364406', 6),
 ('46.9445,7.431886', 6),
 ('49.186934,-2.102182', 6),
 ('48.869512,2.348452', 6),
 ('43.124752,5.929177', 6),
 ('43.

In [24]:
# Visualisation

img = cv2.imread('carte_paris.png')
[w,h,p] = np.shape(img)

def pxInParis():
    nb_px = 0
    for k in range(w):
        for l in range(h):
            if (img[k,l][0] != 132 and img[k,l][1] != 132 and img[k,l][2] != 132):
                nb_px += 1
    return nb_px

nb_px = pxInParis()

km_px = 105.4/nb_px
km_px = m.sqrt(km_px)

arc_de_triomphe = [48.873673, 2.295001]
adt = [137,186]

def gpsTopx(lat,lon):
    dx = conversion(arc_de_triomphe[0],arc_de_triomphe[1],arc_de_triomphe[0],lon)
    dy = conversion(arc_de_triomphe[0],arc_de_triomphe[1],lat,arc_de_triomphe[1])
    px_x,px_y = dx//km_px,dy//km_px
    return adt[0]+px_x,adt[1]-px_y

def poids():
    poids = np.zeros([w,h])
    with open('portails.csv','r') as to_read:
        csv_reader = csv.reader(to_read, delimiter = ';')
        for line in csv_reader:
            lat,lon = float(line[0].split(',')[0]),float(line[0].split(',')[1])
            x,y = gpsTopx(lat,lon)
            if (0<=x<w and 0<=y<h):
                poids[x,y] += 1
    return poids

def zones():
    weight = poids()
    zones = np.ones([w,h])
    for i in range(15):          # l'image fait 500*500px et 1km² = 33px*33px. On cree donc 500/33 = 15 zones par cote
        for j in range(15):
            current_zone = weight[33*i:33*(i+1),33*j:33*(j+1)]
            n,p = np.shape(current_zone)[0],np.shape(current_zone)[1]
            nb_portals = 0
            for k in range(n):
                for l in range(p):
                    nb_portals += current_zone[k,l]
            nb_portals*zones[33*i:33*(i+1),33*j:33*(j+1)]
    return zones

def portalToColour(nb):
    if (nb<5):
        return [0, 0, 50]
    if (nb<10):
        return [0, 0, 100]
    if (nb<15): 
        return [0, 0, 150]
    if (nb<20):
        return [0, 0, 200]
    return [0,0,250]

def visualisation():
    areas = zones()
    for i in range(w):
        for j in range(h):
            if (img[i,j][0] <= 200 and img[i,j][1] <= 200 and img[i,j][2] <= 200):
                img[i,j] = portalToColour(areas[i,j])
    cv2.imwrite('visu.jpg', img)

In [10]:
poids = poids()


/cal/softs/anaconda/anaconda-2.0.1/lib/python2.7/site-packages/IPython/kernel/__main__.py:36: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [16]:
zones = zones()

/cal/softs/anaconda/anaconda-2.0.1/lib/python2.7/site-packages/IPython/kernel/__main__.py:36: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [25]:
visualisation()

/cal/softs/anaconda/anaconda-2.0.1/lib/python2.7/site-packages/IPython/kernel/__main__.py:36: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
